In [1]:
import pandas as pd
import numpy as np
import urllib
import os
import zipfile
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from copy import deepcopy
import tqdm

In [2]:
def make_dir_if_not_present(folder):
    if os.path.exists(folder):
        pass
    else:
        os.mkdir(folder)

In [3]:
make_dir_if_not_present("data")
make_dir_if_not_present("results")
make_dir_if_not_present("figures")
make_dir_if_not_present("scripts")
make_dir_if_not_present("trained_models")
make_dir_if_not_present("studies")

In [4]:
def download_if_not_present(url, destination):
    if os.path.exists(destination):
        pass
    else:
        urllib.request.urlretrieve(url, destination)

In [5]:
# Download GDSC1 and GDSC2
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/GDSC1_fitted_dose_response_27Oct23.xlsx",
                       "data/GDSC1.xlsx")
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/GDSC2_fitted_dose_response_27Oct23.xlsx",
                       "data/GDSC2.xlsx")

# Download compound and cell-line identifiers
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/Cell_Lines_Details.xlsx",
                       "data/Cell_Lines_Details.xlsx")
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/screened_compounds_rel_8.5.csv",
                       "data/screened_compounds_rel_8.5.csv")
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/model_list_20240110.csv",
                       "data/cell_line_annotations.csv")

# Download omics features
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/mutations_all_20230202.zip",
                       "data/raw_mutations.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/rnaseq_all_20220624.zip",
                       "data/rnaseq.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/Proteomics_20221214.zip",
                       "data/proteomics.zip")

download_if_not_present("https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources//Data/BEMs/CellLines/CellLines_METH_BEMs.zip",
                       "data/methylation.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/driver_genes_20221018.csv",
                       "data/driver_genes.csv")
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/driver_mutations_20221208.csv",
                       "data/driver_mutations.csv")

URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
# Preprocessing mutations. 
# All mutations are treated simply as loss of function without accounting for differences, which might not be accurate
mutations = pd.read_csv("data/raw_mutations.zip")
mutations_filtered = mutations.query("coding == True & effect != 'silent'").loc[:, ["gene_symbol", "model_id"]].assign(mutated=1)
mutations_matrix = mutations_filtered.groupby(["model_id", "gene_symbol"]).max().unstack().fillna(0)
mutations_matrix.to_csv("data/binary_mutations.csv")

mutant 데이터를 pre-processing하는 과정이다. 모든 돌연변이를 단순히 "기능 상실"로 취급하고 있으며, 각 모델에서 특정 유전자에 돌연변이가 있는지 여부를 binary 형식으로 변환해 저장한다. 

1. 
   * 여기서 `coding == True`는 coding region에 속하는 mutation만 선택한다. `effect != 'silent'`는 silent mutation이 아닌 경우에만 선택한다.     
   * 이 두가지 필터연산을 통해 mutation만 골라내고, 이어진 `loc[:, ["gene_symbol", "model_id"]]`는 필터링된 데이터에서 `gene_symbol`과 `model_id` column만 선택한다.     
   * 이제 `assign(mutated=1)`를 통해 새로운 열 `mutated`를 추가함과 동시에 값 1을 할당한다. 
2. 
   * `groupby()`는 `model_id`와 `gene_symbol`의 항목 별로 데이터를 그룹화한다. 각 모델에서 특정 유전자에 돌연변이가 발생했는지 여부를 파악하는데에 유용하다. 
   * 이제 이렇게 그룹화된 데이터에서 `.max()`는 각 조합에서의 `mutated`값의 최댓값을 선택한다. 이는 동일한 모델과 유전자에서 여러 돌연변이가 발생할 수 있음을 고려하며, 돌연변이가 발생한 조합만 선택되게된다.    
   * 이후 `.unstack()`은 `model_id`를 row, `gene_symbol`을 col로 하는 매트릭스로 변환하여 나타낸다. 이 결과로 각 모델에 대해 특정 유전자에서 돌연변이가 있었는지를 나타내는 binary matrix가 생성된다. 이렇게 생성된 매트릭스에 NA값이 존재할 수 있는데, `.fillna(0)`를 통해 NA값을 0으로 채워넣는다. 

이렇게 생성된 매트릭스는 `binary_mutation.csv`에 저장된다. 


In [ ]:
# Preprocessing proteometics
zipf = zipfile.ZipFile("data/proteomics.zip")
zipf.extractall("data/")
protein_intensities = pd.read_csv("data/proteomics_all_20221214.csv").groupby([ "model_id", "uniprot_id"])["protein_intensity"].median().unstack()

In [ ]:
protein_filtered = protein_intensities.loc[:, protein_intensities.isna().sum(0)< 10] # filter proteins that have a low number of missing values
# 결측치가 10개 이하인 protein만 사용

In [ ]:
protein_filtered

In [ ]:
proteins = protein_filtered.columns

In [ ]:
protein_input = deepcopy(protein_filtered)

In [ ]:
# Note that for this inputation the pipeline leaks at several points
protein_filtered_copy = deepcopy(protein_filtered)
protein_input = deepcopy(protein_filtered)
medians = protein_filtered.median()
protein_filtered_copy = protein_filtered.fillna(medians)

for protein in tqdm.tqdm(proteins): 
    target_inputation = protein_filtered.loc[:, protein] # 1
    if target_inputation.isna().sum() > 0: # 2
        X = protein_filtered_copy.loc[~target_inputation.isna()]
        X_input = protein_filtered_copy.loc[target_inputation.isna()]
        y = X.loc[:, protein] # 3
        X = X.drop(protein, axis=1)
        grid = GridSearchCV(estimator = Ridge(), # 4
                             param_grid = {"alpha": [0.1, 1, 100, 200, 500, 1000, 5000, 10000]},
                             scoring = "r2",
                             n_jobs = -1)
        grid.fit(X, y)
        rdg = Ridge(**grid.best_params_) # 5
        rdg.fit(X, y)
        y_input = rdg.predict(X_input.drop(protein, axis=1)) # 6
        protein_input.loc[target_inputation.isna(), protein] = y_input
protein_input.to_csv("data/proteomics.csv") # 7

단백질 데이터를 기반으로 결측값을 보완(임퓨테이션)하는 과정에서 Ridge 회귀 모델을 사용한 방식입니다. 전체적인 흐름은 결측값을 포함한 데이터를 사용해 결측값을 예측하는 모델을 학습하고, 예측된 값을 채워넣는 방식으로 구성되어 있습니다.

`medians = protein_filtered.median()`는 각 열에 대해 중앙값을 계산하고, `protein_filtered_copy`의 결측값을 중앙값으로 채워넣는다. 

`tqdm`은 파이썬에서 반복문 실행 시 progress를 표시해주는 라이브러리다. 작업의 진행률을 실시간으로 표시해준다.

1. 각 단백질에 대해, `target_inputation = protein_filtered.loc[:, protein]`은 현재 단백질 열(col)을 선택하여 가져온다. 
2. 만약 선택된 행에 결측값이 하나라도 존재하면(`target_inputation.isna().sum() > 0`), 결측값이 없는 행을 X에(`protein_filtered_copy.loc[~target_inputation.isna()]`), 있는 행을 `X_input`에 넣는다(`protein_filtered_copy.loc[target_inputation.isna()]`).
3. y는 해당 단백질의 값(결측값 제외!)을 저장하여 결측값 예측을 위해 종속변수로 사용되고, X는 현재 단백질을 제외한 나머지 단백질 데이터들로 독립변수로 사용된다. 
4. `GridSearchCV`를 사용해 Ridge Regression model에 대한 최적의 하이퍼파라미터 알파를 찾는다. 주어진 리스트에서 최적의 값을 선택한다. 성능평가는 r2로 측정한다. 이는 객체 `grid`에 대한 `fit`으로 실행된다. 다양한 알파를 가지고 여러번 모델을 학습하며, 가장 좋은걸 선택한다. 
5. `grid`가 찾아낸 최적의 파라미터를 이용해서 Ridge 모델인 `rdg`를 학습시킨다.
6. 이후 결측값이 있는 단백질에 대해, rdg로 예측을 수행하여 채워넣는다. 
7. 반복 후, 저장! 


In [ ]:
# Preprocessing methylation
zipf = zipfile.ZipFile("data/methylation.zip")
zipf.extractall("data/")
pd.read_csv("data/METH_CELLLINES_BEMs/PANCAN.txt", sep = "\t").to_csv("data/methylations.csv")

In [ ]:
# Preprocessing expression
zipf = zipfile.ZipFile("data/rnaseq.zip")
zipf.extractall("data/")

압축파일을 전부 풀어준다.

In [ ]:
rnaseq = pd.read_csv("data/rnaseq_read_count_20220624.csv")

In [ ]:
rnaseq = rnaseq.iloc[4:, 1:].set_index("Unnamed: 1")

1-3번 행은 데이터가 아닌 정보를 포함하고 있기 때문에, 4번 행부터 사용하고, 0번 열은 gene id를 나타내기 때문에, 1번 열부터 뜯어온다. 또한, 첫번째 열을 index로 설정한다. 

- `iloc`은 인덱싱 함수


In [ ]:
rna_filtered = rnaseq.loc[rnaseq.isna().sum(1) == 0] # we remove genes with nans

결측값이 없는 유전자만 남기는 필터링 작업. `isna().sum(1) == 0`는 각 행에 결측값이 없는 애들을 의미한다. 이런애들만 뽑아 모아서 filtered에 저장한다. 

In [ ]:
rna_filtered = rna_filtered.replace(" [0]*", "", regex = True)

regex를 이용하여 불필요한 패턴을 제거한다. 

In [ ]:
rna_filtered = (rna_filtered.T).astype(float)

원활한 수치 계산을 위해 float형으로 바꿔준 뒤, transpose한다. 

In [ ]:
rna_log = np.log(rna_filtered + 1)

로그 변환을 하는데, 이때 1을 더해서 0에 대한 로그변환을 예방한다. 0은 로그변환을 할 수 없기 때문임. 정의가 안됌

In [ ]:
rna_count_norm = (rna_log - rna_log.mean(0))/rna_log.std(0) # by machine learning standards data leakage

z-score 정규화. 이는 데이터 누출을 방지하는 일반적인 규칙이다. 

In [ ]:
rna_count_norm.dropna(axis=1).to_csv("data/rnaseq_normcount.csv") # remove columns with zero variance and save

`dropna(axis=1)`를 통해 결측값을 포함한 열을 제거한다. 이전 과정에서 분산이 0인 애들이 na로 저장되는 듯 하다. 확실하진 않음.

암튼 분산이 0인 애들은 모든 데이터가 같아 정보가 없으므로 제거하는 것이 일반적이다. 

In [ ]:
GDSC1 = pd.read_excel("data/GDSC1.xlsx")
GDSC1.loc[:, ["SANGER_MODEL_ID", "DRUG_ID", "LN_IC50"]].to_csv("data/GDSC1.csv")

GDSC 데이터에서 필요한 칼럼만 가져와서 따로 저장한다. 